In [1]:
from langchain_openai import ChatOpenAI
import PyPDF2 as pypdf
import docx
import json

In [2]:
llm = ChatOpenAI(
    openai_api_key= "EMPTY",
    openai_api_base = "http://10.179.82.226:8010/v1",
    model_name = "./models/Meta-Llama-3.1-8B-Instruct-awq",
    temperature = 0,  
    max_tokens = 2000,
    stop_sequences=["<|eot_id|>"],
    # model_kwargs={"stop": ["<|eot_id|>"]},
)

In [3]:
from langchain_core.messages import SystemMessage, HumanMessage

system_message = SystemMessage("You are a poetic person")
human_message = HumanMessage("Who is the prettiest person")
llm.invoke([system_message, human_message])

AIMessage(content="My friend, the concept of beauty is like a moonlit night, shrouded in mystery and subjective to the beholder's gaze. What one person finds breathtakingly beautiful, another might not even notice.\n\nIn the realm of the human form, I'd argue that beauty lies not just in the physical, but in the essence that radiates from within. It's the sparkle in a person's eyes, the gentle curve of a smile, the softness of a whispered word.\n\nIf I had to choose, I'd say that the prettiest person is the one who embodies the qualities of a sunrise – warm, vibrant, and full of promise. Someone who radiates kindness, compassion, and a sense of wonder, like a dew-kissed flower that blooms in the morning's golden light.\n\nBut, alas, beauty is a fleeting dream, a whispered promise of the soul. It's the whispered secrets, the laughter, and the tears that make a person truly beautiful. It's the way they make you feel, like a gentle breeze on a summer's day, or the warmth of a crackling fi

In [4]:
def extract_text_from_pdf(file):
    text = ""
    if file.endswith(".pdf"):
        
        pdf_reader = pypdf.PdfReader(file)
        # obtain the text from the pdf
        #text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
    elif file.endswith(".docx"):
        doc = docx.Document(file)
        #text = ""
        for para in doc.paragraphs:
            text += para.text
    elif file.endswith(".txt"):
        #text=""
        with open(file, "r") as f:
            text = f.read()
    return text

In [5]:
def extract_single_json_object(jsonString: str):
    firstValue = jsonString.index("{")
    lastValue = len(jsonString) - jsonString[::-1].index("}")
    jsonString = jsonString[firstValue:lastValue]

    return jsonString

In [6]:
def extract_jd_info(jd_text):
    prompt = f"""
Extract the following information from the job description :
- job_role (Job role for which interview is being conducted.)  
- educational_qualification (Educational Qualification needed for the job role.)  
- work_experience (Minimum years needed. Return empty if no such information is provided.)
- must_have_skills (Only Skills or technology which are specifically mentioned as must have in the text provided.)
- optional_skills (Skills or any knowledge which are specifically mentioned as optional to have or good to have in the text provided.)

EXPECTED OUTPUT: Json Object with the extracted details. If any detail is not present return an empty string.

JOB DESCRIPTION:
{jd_text}
"""
    
    response = llm.invoke(prompt).content
    # print(f"response:{response}")
    # print(f"EXTRACTED: {extract_single_json_object(response)}")
    response = eval(extract_single_json_object(response))

    return response

In [7]:
def extract_resume_info(resume_text):
    prompt = f"""
Extract the following information from the job description :
- name (Name of the candidate)
- email (Email of the candidate)
- education_qualification (Education/Qualification of the candidate along with CGPA/Percentage)
- work_experience (Professional Work Experience in enterprises including time duration in years with all tasks and responsibilities carried out. Do not include projects in this field.)
- skills (Any and all tools or softwares or technical knowledge used or mentioned by the candidate anywhere in the context including in projects)
- project_tools (tools used in projects. Return empty list if none mentioned)

EXPECTED OUTPUT: Json Object with the extracted details. If any detail is not present return an empty string. Ensure all numbers, fraction are put in double quotes.

JOB DESCRIPTION:
{resume_text}
"""
    response = llm.invoke(prompt).content
    # print(f"ISSUE CAUSING RESPONSE{response}")
    response = eval(extract_single_json_object(response))

    return response

In [34]:
jd_path = "JDs\\Senior Software Engineer Job Description 1.pdf"
resume_path = "Resumes\\ANSHUL RESUME.pdf"

jd_text = extract_text_from_pdf(jd_path)
resume_text = extract_text_from_pdf(resume_path)

In [35]:
jd_info = extract_jd_info(jd_text)
resume_info = extract_resume_info(resume_text)

In [36]:
jd_info["optional_skills"]

[]

In [38]:
for i in range(1):
    print(f"ITERATION: {i}") 
    skills_match = {}
    Resume_Skills = list(set(resume_info['skills'] + resume_info["project_tools"]))
    for skill in jd_info["must_have_skills"]:
        prompt = f"""
Perform the following steps:
RESUME SKILLS of the Candidate are: {Resume_Skills}
1. For the JD skill "{skill}" identify the resume skills if it is explicitly and exactly matches otherwise ignore. DO NOT CONSIDER BROADER MATCHES OR ASSUME SIGNI.
2. create only a single json object with the following structure:
{{
    "{skill}": list of exactly matched skills
}}
    
"""
        resp = llm.invoke(prompt).content
        print("$$$$$$$   OP1",resp)

        prompt2 = f"""
Extract the final json from the below output. if no json is provided then return: {{{skill} : []}}
###OUTPUT###
{resp}
"""
        response = llm.invoke(prompt2).content
        print("$$$$$$$   IP2 ",response)
        skills_match.update(json.loads(f"{extract_single_json_object(response)}"))
        # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        # print("$$$$$$$   OP",response)

    print(skills_match)

ITERATION: 0
$$$$$$$   OP1 Here is the JSON object with the exactly matched skills for the JD skill "JavaScript":

```
{
  "JavaScript": ["JavaScript"]
}
```

Since "JavaScript" is explicitly mentioned in the resume skills, it is the only match.
$$$$$$$   IP2  Here is the final JSON output:

```
{
  "JavaScript": ["JavaScript"]
}
```

Let me know if you'd like me to help with anything else!
$$$$$$$   OP1 Based on the provided resume skills, the JD skill "LAMP stacks" exactly matches the following resume skills:

* PHP
* CSS
* HTML

Here is the JSON object with the exact matches:

```
{
  "LAMP stacks": ["PHP", "CSS", "HTML"]
}
```

Note that I ignored Bootstrap as it is a broader match (a CSS framework) and not an exact match for "LAMP stacks".
$$$$$$$   IP2  The final JSON output is:

```
{
  "LAMP stacks": ["PHP", "CSS", "HTML"]
}
```

Let me know if you'd like me to help with anything else!
$$$$$$$   OP1 Based on the provided resume skills, here is the JSON object with the exact mat

In [48]:
for i in range(1):
    print(f"ITERATION: {i}") 
    skills_match = {}
    Resume_Skills = list(set(resume_info['skills'] + resume_info["project_tools"]))
    for skill in jd_info["must_have_skills"]:
        prompt = f"""
For the following JD skill requirement, match the skills, ONLY from Resume Skills provided, which satisfy the requirement under consideration. If the JD Requirement is directly and explicitly mentioned in the Resume Skills, only then return the same skill. Be extremely strict and hard to please in your judgement.

###JD SKILL REQUIRED###
{skill}
### RESUME SKILLS PROVIDED ###
{Resume_Skills}

return the result as a json with jd skill as key and the matched skills as value. If no match found, return an empty list.
"""
        resp = llm.invoke(prompt).content

        prompt2 = f"""
Extract the final json from the below output
###OUTPUT###
{resp}
"""
        response = llm.invoke(prompt2).content
        # print("$$$$$$$   OP INPUT ",response)
        skills_match.update(json.loads(f"{extract_single_json_object(response)}"))
        # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        # print("$$$$$$$   OP",response)

    print(skills_match)

ITERATION: 0
{'Python': ['PYTHON'], 'TensorFlow/PyTorch': [], 'scikit-learn': [], 'pandas': ['PYTHON'], 'numpy': ['PYTHON'], 'matplotlib': [], 'API development and programming using Python': ['PYTHON'], 'experience with Python Flask': [], 'knowledge of RDBMS concepts (MySQL/Oracle/MongoDB)': ['SQL', 'DBMS', 'MongoDB'], 'Understanding of various machine learning algorithms for predictive and segmentation analysis': ['MACHINE LEARNING', 'PYTHON'], 'Proficiency in various Python libraries': ['PYTHON'], 'Perform data analysis and algorithm development to drive decision-making and business intelligence': ['MACHINE LEARNING', 'PYTHON', 'SQL', 'DBMS', 'MongoDB'], 'Collaborate with other team members to evaluate data, suggest solutions, and improve the accuracy and speed of existing models': [], 'Prototype new deep learning algorithms and technologies': [], 'Work closely with cross-functional teams to integrate AI solutions into existing systems and processes': [], 'Work on staying up-to-date 

In [34]:
def matchRequirements_os(jd_info, resume_info):
    skills_match = {}
    Resume_Skills = list(set(resume_info['skills'] + resume_info["project_tools"]))
    for skill in jd_info["optional_skills"]:
        prompt = f"""
For the following JD skill requirement, match the skills, ONLY from Resume Skills provided, which satisfy the requirement under consideration. 

Be very precise while matching the skills to the JD-Requirement and only consider the skill if it completely satisfies the requirement. 

###JD SKILL REQUIRED###
{skill}
### RESUME SKILLS PROVIDED ###
{Resume_Skills}
return the result as a json with format:
"JD-Requirement":["Resume skill1", "Resume skill2"]
where JD-Requirement is the key.
"""
        resp = llm.invoke(prompt).content

        prompt2 = f"""
Extract the final json from the below output
###OUTPUT###
{resp}
"""
        response = llm.invoke(prompt2).content
        # print("$$$$$$$   OP INPUT ",response)
        skills_match.update(json.loads(f"{extract_single_json_object(response)}"))
        # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        # print("$$$$$$$   OP",response)

    return skills_match

In [35]:
for i in range(5):
    print(f"ITERATION: {i}") 
    skills_match = matchRequirements_os(jd_info, resume_info)
    print(skills_match)

ITERATION: 0
{'Knowledge of any leading cloud platform like Microsoft Azure, Amazon Web Service, Google Cloud': [], 'Good understanding of complex event analysis, advanced analytics etc.': ['MACHINE LEARNING', 'PYTHON', 'SQL', 'DBMS'], 'Hands-on experience on projects and internships': ['OOPS', 'COMPETITIVE PROGRAMMING', 'GIT', 'REACT', 'NEXTJS', 'PYTHON', 'Node.js', 'SQL', 'MongoDB', 'HTML/CSS', 'C++']}
ITERATION: 1
{'Knowledge of any leading cloud platform like Microsoft Azure, Amazon Web Service, Google Cloud': [], 'Good understanding of complex event analysis, advanced analytics etc.': ['MACHINE LEARNING', 'PYTHON', 'SQL', 'DBMS'], 'Hands-on experience on projects and internships': ['OOPS', 'COMPETITIVE PROGRAMMING', 'GIT', 'REACT', 'NEXTJS', 'PYTHON', 'Node.js', 'SQL', 'MongoDB', 'HTML/CSS', 'C++']}
ITERATION: 2
{'Knowledge of any leading cloud platform like Microsoft Azure, Amazon Web Service, Google Cloud': [], 'Good understanding of complex event analysis, advanced analytics et

In [7]:
def calculateQualificationScore(jd_info, resume_info):
    #Calculating the score
    prompt = f"""
Educational qualification required for the job are {', '.join(jd_info['educational_qualification'])}
#########################
Education and other qualifications of the candidate are {', '.join(resume_info['education_qualification'])}

Score the qualification of the candidate with the job description such as education, work experience, etc between 0 and 1 in decimals. 
Be lenient while grading and even if educational degrees are similar give 1.
"""
    # print("#######################")
    # print(prompt)
    # print("#######################")

    response = llm.invoke(prompt)
    # print(response)

    return response.content


In [8]:
def calculateWorkExperienceScore(jd_text, resume_text):
    work_experience = "of 0 years" if not resume_text['work_experience'] else ', '.join(resume_text['work_experience'])
    jd_relevant_text = f""" 
Work Experience needed for the job is {', '.join(jd_text['work_experience'])} 
Must have skills are {', '.join(jd_text['must_have_skills'])}
"""
    resume_relevant_text = f""" 
Education and other qualifications of the candidate are {', '.join(resume_text['education_qualification'])}
Work Experiences of the candidate are {work_experience}
"""
    #Calculating the score
    prompt = f"""
```{jd_relevant_text}```
#########################
```{resume_relevant_text}```

Follow the following steps to calculate the work experience score:
1. For all the experiences mentioned, determine whether they are only project experiences and reject them if they are.
2. For all the remaining experiences, shortlist only those that mentioned that they utilized the needed skills.
3. Now for all the remaining relevant experiences calculate or extract the total number of months of experience by subtracting the start and end date if provided for all relevant instances and sum them all up. Take Present year as 2024 if needed for calculating experience
4. Calculate the score as a ratio of (candidate's relevant experience in months/ work experience required in months) with MAXIMUM VALUE as 1.
5. Remember that the score should have a maximum value of 1.
6. If there is no work experience mentioned, give a score of 0.
"""
    # print("#######################")
    # print(prompt)
    # print("#######################")

    response = llm.invoke(prompt)
    # print(response)

    return response.content

In [9]:
def matchRequirements_ms(jd_text, resume_text):
    JD_skills = jd_text['must_have_skills'] 
    # print(f'{JD_skills} JD Skills')

    Resume_Skills = resume_text['skills'] + resume_text["project_tools"]
    #Calculating the score
    prompt = f"""Step by Step for each JD Requirement match the skills, ONLY from Resume Skills provided, which satisfy the requirement under consideration
###JD REQUIREMENT###
{JD_skills}
###RESUME SKILLS###
{Resume_Skills}

step 1: For each JD requirement, consider all resume skills and create the list of resume skills that satisfy the requirement. Be strict while matching the must have skills.
step 2: return the result as a json with format:
"JD-Requirement":["Resume skill1", "Resume skill2"]
where JD-Requirements are keys.
"""
    # print("Requirements Score#######################")
    # print(prompt)
    # print("Requirements Score#######################")

    resp = llm.invoke(prompt).content

    prompt2 = f"""
Extract the final json from the below output
###OUTPUT###
{resp}
"""
    response = llm.invoke(prompt2).content
    response = json.loads(f"{extract_single_json_object(response)}")
    # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    # print("$$$$$$$   MS",response)
    # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    return response

In [10]:
def matchRequirements_os(jd_text, resume_text):
    JD_skills = jd_text['optional_skills']

    Resume_Skills = list(set(resume_text['skills'] + resume_text["project_tools"]))
    #Calculating the score
    prompt = f"""Step by Step for each JD Requirement match the skills, ONLY from Resume Skills provided, which satisfy the requirement under consideration
###JD REQUIREMENT###
{JD_skills}
###RESUME SKILLS###
{Resume_Skills}

step 1: For each JD requirement, consider all resume skills and create the list of resume skills that satisfy the requirement. Be strict while matching the optional skills.
step 2: return the result as a json with format:
"JD-Requirement":["Resume skill1", "Resume skill2"]
where JD-Requirements are keys.
"""
    # print("Requirements Score#######################")
    # print(prompt)
    # print("Requirements Score#######################")

    resp = llm.invoke(prompt).content

    prompt2 = f"""
Extract the final json from the below output
###OUTPUT###
{resp}
"""
    response = llm.invoke(prompt2).content
    response = json.loads(f"{extract_single_json_object(response)}")
    # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    # print("$$$$$$$   MS",response)
    # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    return response

In [11]:
def extract_qual_work_score(combined_eval: str):
    prompt = f"""
Extract the following individual scores of the candidates from the evaluation provided:
- qualification_score (Score of the qualification of the candidate with the job description such as education etc. If no such information is provided always give 0.)

- work_experience_score (Score of the work experience of the candidate with the job description. If no such information is provided always give 0.)

EXPECTED OUTPUT: Json Object with the extracted details

EVALUATION:
{combined_eval}
"""
    response = llm.invoke(prompt).content
    response = eval(extract_single_json_object(response))

    return response


In [12]:
jd_text = extract_text_from_pdf("JD - ML.pdf")
jd_dict = extract_jd_info(jd_text)


In [13]:
jd_dict

{'job_role': 'Machine Learning Intern',
 'educational_qualification': 'B.Tech / BE in Computer Science Engineering / Electronics / IT Engineering with CGPA 6.5 OR 65% and above',
 'work_experience': '',
 'must_have_skills': ['Develop and test deep learning models using Python, TensorFlow/PyTorch, and other open-source frameworks',
  'Write clean, maintainable code for AI applications',
  'Understanding of various machine learning algorithms for predictive and segmentation analysis',
  'Proficiency in various Python libraries such as scikit-learn, pandas, numpy, matplotlib etc.',
  'Perform data analysis and algorithm development to drive decision-making and business intelligence',
  'Collaborate with other team members to evaluate data, suggest solutions, and improve the accuracy and speed of existing models',
  'Prototype new deep learning algorithms and technologies',
  'Work closely with cross-functional teams to integrate AI solutions into existing systems and processes',
  'Work o

In [14]:
resume_text = extract_text_from_pdf("Resume 2.pdf")
resume_dict = extract_resume_info(resume_text)


In [15]:
resume_dict

{'name': 'YASHITA BANSAL',
 'email': 'yashitabansal39@gmail.com',
 'education_qualification': {'B.TECH IN COMPUTER SCIENCE': {'university': 'PUNJAB ENGINEERING COLLEGE',
   'duration': '12/2021 - 06/2025',
   'cgpa': 9.49},
  'CLASS-XII': {'school': 'ISHWAR SINGH DEV SAMAJ SENIOR SECONDARY SCHOOL',
   'duration': '05/2019 - 05/2021',
   'percentage': 89},
  'CLASS - X': {'school': 'LITTLE FLOWER CONVENT SCHOOL',
   'duration': '03/2007 - 04/2019',
   'percentage': 97.6}},
 'work_experience': [],
 'skills': ['COMPETITIVE PROGRAMMING',
  'PYTHON',
  'C++',
  'HTML/CSS',
  'JAVASCRIPT',
  'SQL',
  'NEXTJS',
  'TAILWIND CSS',
  'REACT',
  'GIT',
  'DBMS',
  'OOPS',
  'MACHINE LEARNING'],
 'project_tools': ['NEXT.js', 'Tailwind CSS', 'Node.js', 'MongoDB']}

In [16]:
score_tracking = {}


In [17]:
if not jd_dict['work_experience']:
    score_tracking["work_experience"] = "NA"
if not jd_dict['educational_qualification']:
    score_tracking["education_qualification"] = "NA"
if not jd_dict['optional_skills']:
    score_tracking["optional_skills"] = "NA"

In [18]:
denom = 0
combined_eval = ""
if not score_tracking.get('work_experience'):
    work_exp_eval = calculateQualificationScore(jd_dict, resume_dict)
    combined_eval += f"{work_exp_eval}\n"
    denom += 4
if not score_tracking.get('education_qualification'):
    edu_eval = calculateQualificationScore(jd_dict, resume_dict)
    combined_eval += f"{edu_eval}\n"
    denom += 1
if not score_tracking.get('optional_skills'):
    denom += 2
    satisfied_opt_count = 0
    req_opt_eval = matchRequirements_os(jd_dict, resume_dict)
    for key in req_opt_eval:
        if not (len(req_opt_eval[key]) == 0):
            satisfied_opt_count += 1

denom += 4
satisfied_must_count = 0
req_must_eval = matchRequirements_ms(jd_dict, resume_dict)
for key in req_must_eval:
    if not (len(req_must_eval[key]) == 0):
        satisfied_must_count += 1

if combined_eval != "":
    qual_score_dict = extract_qual_work_score(combined_eval)
else:
    qual_score_dict = {"qualification_score": 0, "work_experience_score": 0}

if not score_tracking.get('optional_skills'):
    value=4*(satisfied_must_count/len(jd_dict["must_have_skills"]))+2*(satisfied_opt_count/len(jd_dict["optional_skills"]))
else:
    value=4*(satisfied_must_count/len(jd_dict["must_have_skills"]))

final_result = (qual_score_dict['qualification_score'] + 4*min(qual_score_dict['work_experience_score'], 1) + value)/denom*100


In [19]:
final_result, qual_score_dict['qualification_score'], qual_score_dict['work_experience_score'], value, denom

(69.6969696969697, 1.0, 0.0, 3.878787878787879, 7)

In [20]:
satisfied_must_count, satisfied_opt_count

(7, 2)

In [21]:
req_opt_eval

{'Good understanding of Linux': ['GIT'],
 'Hands-on experience in code versioning tools such as Git': ['GIT'],
 'Strong technical acumen and understanding of emerging disruptive technologies, like cloud computing, IoT, complex event analysis, advanced analytics etc.': []}

In [1]:
json_value = """
{
    "name": "Kriti Mahajan",
    "email": "kriti21.2002@gmail.com",
    "education_qualification": {
        "Degree": "B. Tech.",
        "Field of Study": "Electronics & Communication",
        "University": "Punjab Engineering College, Chandigarh",
        "CGPA": 9.11/10,
        "Percentage": 98.4%
    },
    "work_experience": [],
    "skills": [
        "Microsoft Office - Word, Excel, PowerPoint",
        "SQL (basics)",
        "Programming - C++ , Python(basics), Data Structures and Algorithms",
        "HTML, CSS",
        "Leadership and Team Management Skills",
        "Elocution and Communication Skills",
        "Analytical Skills"
    ],
    "project_tools": [
        "Priority Queues",
        "Hashmaps",
        "Binary Tree"
    ]
}
"""

In [3]:
import json

data = json.loads(json_value)

JSONDecodeError: Expecting ',' delimiter: line 9 column 21 (char 275)